Portfolio Check
=====================

An interactive OpenQuake (OQ) tool to build a portfolio of assets and check their risk. Written by Tiegan Hobbs ([tieganh](https://github.com/tieganh)) on 11 May 2020 for python. This document is intended for demonstration purposes, and licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/). 
![alt text](https://i.creativecommons.org/l/by-sa/4.0/80x15.png "CC BY-SA 4.0")

------------

## Table of Contents 

1. [Build Portfolio](#Build-Portfolio)
2. [Run Scenario](#Run-Scenario)
3. [View Report](#View-Report)


----------
---------

## Running OpenQuake
To run this tutorial you will need to have OQ installed. Test it by running the code below to ensure you get a recent version (ex: '3.9.0-gitdcd091b') rather than an error. Assuming it works, run `oq webui start` in a terminal window to initialize the OQ web interface. If you encounter errors, see the [installation guide](https://github.com/gem/oq-engine/blob/master/doc/installing/development.md).

In [ ]:
%%bash
oq --version

-----------

## Building Portfolio

Run the code snippet below, and drag the marker to your asset of concern.

In [67]:
from ipyleaflet import Map, Marker

center = (49.2609, -123.0924)
m = Map(center=center, zoom=15)
marker = Marker(location=center, draggable=True)
m.add_layer(marker);

display(m)

Map(center=[49.2609, -123.0924], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

**Run the code below to save your new location.**

In [68]:
LAT, LON = marker.location
print(LAT,LON)

49.265468188307544 -123.09197131544352


Select your construction type and occupancy from the dropdowns below.

In [86]:
import ipywidgets as widgets
Drop1 = widgets.Dropdown(
    options=['Wood', 'Steel', 'Concrete', 'Precast', 'Reinforced Masonry', 'Unreinforced Masonry', 'Mobile Home'],
    description='Material:',
)
Drop1

Dropdown(description='Material:', options=('Wood', 'Steel', 'Concrete', 'Precast', 'Reinforced Masonry', 'Unre…

In [87]:
if Drop1.value is 'Wood':
    Storeys = widgets.IntSlider(
        description='Storeys:',
        value=2,
        min=1,
        max=3,
        step=1,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d')
    print("Choose number of storeys. If greater than 3, select 3.")
else:
    Storeys = widgets.IntSlider(
        description='Storeys:',
        value=2,
        min=1,
        max=8,
        step=1,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d')
    print("Choose number of storeys. If greater than 8, select 8.")

Storeys

Choose number of storeys. If greater than 8, select 8.


IntSlider(value=2, continuous_update=False, description='Storeys:', max=8, min=1)

In [88]:
if Drop1.value is 'Wood':
    Drop2 = widgets.Dropdown(
        options=[('Light [<5000 sq ft]','W1'),('Commercial/Industrial [>5000 sq ft]','W2')],
        description='Type:',
    )
elif Drop1.value is 'Unreinforced Masonry':
    
elif Drop1.value is 'Concrete':
    Drop2 = widgets.Dropdown(
        options=[('Moment Frame','C1'),('Shear Wall','C2'), ('URM Infill','C3')],
        description='Type:',
    )
Drop2

Dropdown(description='Type:', options=(('Moment Frame', 'C1'), ('Shear Wall', 'C2'), ('URM Infill', 'C3')), va…

In [89]:
if Storeys.value in (1,2,3):
    

if Drop1.value is 'Concrete':
    CONST=str(Drop2.value)+str(Storeys.value)
elif Drop1.value is 'Wood':
    CONST = Drop2.value
    
print(CONST)

C36


In [40]:
from ipywidgets import interact, interact_manual
@interact
def building_material(BuildingMaterial=['wood', 'concrete', 'URM', 'steel']):
    if BuildingMaterial is 'wood':
        def wood_type(CONST=[('Commercial/Industrial','W2'), ('Light Frame','W1')]):
            return CONST
    else:
        return "NOT WOOD"
    

interactive(children=(Dropdown(description='BuildingMaterial', options=('wood', 'concrete', 'URM', 'steel'), v…

In [25]:
OCC="RES1"
print(CONST)

w1


You'll also need to specify the number of people who reside in your structure. That's the 'nighttime' population. Based on the number of people who work outside of the home, you will then specify a 'daytime' population as well as a 'transit' population. The latter refers to the number who would be at home during transit hours, and is usually somewhere between daytime and nighttime population. If you don't know exactly, just do your best. 

In [ ]:
NIGHT="6"
DAY="3"
TRANSIT="5"
echo $DAY $NIGHT $TRANSIT

Finally, you'll need to decide on your seismic code level. This depends on the year that your structure was built (or retrofitted) as well as the design level at your site. The latter depends on your seismic hazard, which is expressed in terms of the Spectral Acceleration (SA) with a 2% in 50 year probability of exceedance, for long (Sl) and short (Ss) periods. You can find out the seismic hazard for your location with the [Earthquakes Canada Seismic Hazard Calculator](https://earthquakescanada.nrcan.gc.ca/hazard-alea/interpolat/calc-en.php). 

1. Enter your latitude and longitude, leaving all other fields at default. Hit 'Submit'.
2. In the 2% in 50 year chart (top result), under a distance of 0 km, find your Sa(0.2) and Sa(1.0). 
3. Use those values with the chart below, as well as the year of construction/retrofit, to determine your code level.

					
|UBC Seismic Zone (NEHRP Map Area)|Sa0.2|Sa1.0|2005-present|1990-2005|1973-1990|Pre-1973|
|:--------------------------------|:----|:----|:-----------|:--------|:--------|:-------|
|Zone 4 (Map Area 7)|SS > 1.500g|S1 > 0.600g|High-Code|High-Code|Moderate-Code|Pre-Code|
|Zone 3 (Map Area 6)|1.000g < SS < 1.500g|0.400g < S1 < 0.600g|High-Code|Moderate-Code|Moderate-Code|Pre-Code|		
|Zone 2(Map Area 4/5)|0.500g < SS < 1.000g|0.200g < S1 < 0.400g|Moderate-Code|Moderate-Code|Low-Code|Pre-Code|		
|Zone 1 (Map Area 2/3)|0.250g < SS < 0.500g|0.100g < S1 < 0.200g|Low-Code|Low-Code|Pre-Code (W1=Low)|Pre-Code|
|Zone 0 (Map Area 1)|SS < 0.250g |S1 < 0.100g |Low-Code |Pre-Code (W1=Low)|Pre-Code (W1=Low)|Pre-Code|

4. Specify your code level in the prompt below, using acronyms.    
    HC = High-Code    
    MC = Mid-Code   
    LC = Low-Code    
    PC = Pre-Code

In [ ]:
CODE="LC"
echo $CODE

Now you can combine the features of your building into a csv exposure file. The code below is just printing all the information we've input so far into a new csv file called `exposure.csv`, formatted correctly for OQ. You can change the name or location in the first line under `FILE_OUTPUT`. Note that it will replace any pre-existing file with the same name in this location.

In [ ]:
CSV_OUTPUT="./exposure.csv"

TAXO=`echo $OCC $CONST $CODE | awk 'BEGIN {OFS="-"} {print $1,$2,$3}'`
ID="1"; NUM="1"
echo "id,lon,lat,taxonomy,number,day,night,transit,csdname"
echo $ID $LON $LAT $TAXO $NUM $DAY $NIGHT $TRANSIT $MUNI | awk 'BEGIN {OFS=","} {print $1,$2,$3,$4,$5,$6,$7,$8,$9}'
echo "id,lon,lat,taxonomy,number,day,night,transit,csdname" > ${CSV_OUTPUT}
echo $ID $LON $LAT $TAXO $NUM $DAY $NIGHT $TRANSIT $MUNI | awk 'BEGIN {OFS=","} {print $1,$2,$3,$4,$5,$6,$7,$8,$9}' >> ${CSV_OUTPUT}

*Note that if you wanted to add extra buildings to your code then you could update the location, construction, occupant, and code-level variables in the appropriate snippets and run the following command to append your output csv file:*

In [ ]:
TAXO=`echo $OCC $CONST $CODE | awk 'BEGIN {OFS="-"} {print $1,$2,$3}'`
echo $ID $LON $LAT $TAXO $NUM $DAY $NIGHT $TRANSIT $MUNI | awk 'BEGIN {OFS=","} {print $1,$2,$3,$4,$5,$6,$7,$8,$9}'
echo $ID $LON $LAT $TAXO $NUM $DAY $NIGHT $TRANSIT $MUNI | awk 'BEGIN {OFS=","} {print $1,$2,$3,$4,$5,$6,$7,$8,$9}' >> ${CSV_OUTPUT}

Now we will build the xml wrapper for our exposure file.

In [ ]:
XML_OUTPUT="./exposure.xml"

cat > exposure.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<nrml xmlns="http://openquake.org/xmlns/nrml/0.4">
    <exposureModel id="ex1" category="buildings" taxonomySource="GEM taxonomy">
        <description></description>
        <conversions>
            <costTypes>
            </costTypes>
        </conversions>
        <occupancyPeriods>day night transit</occupancyPeriods>
        <tagNames>csdname</tagNames>
    <assets>  
        exposure.csv
    </assets>
    </exposureModel>
</nrml>
EOF

----------------------------
### Building An Earthquake Rupture

Now we need an earthquake to shake the house with. In a future version, this tutorial can be used to build your own rupture. For now, we'll just call to the pre-existing Georgia Strait M7.3 scenario. It's included in the `what_you_need/` directory that accompanies this tutorial. 

In [ ]:
RUPTURE="what_you_need/rupture_AFM7p3_GSM.xml"

-------------

### Picking Soil Conditions
Likewise, a future version of this tutorial will allow you to resample the National database files of site conditions (ex: `vs30_BC_site_model.csv`) to obtain a realistic estimate for your house. In the meantime, we'll use a value of 450 m/s to approximate stiff soil or loose rock (Site Class C).

|Site Class|Profile Name|Vs30 (m/s)|
|:--------:|:-----------|:---------|
|A|Hard rock|Vs30 > 1500|
|B|Rock|760 < Vs30 ≤ 1500|
|C|Very dense soil and soft rock|360 < Vs30 ≤ 760|
|D|Stiff soil|180 < Vs30 ≤ 360|
|E|Soft soil|Vs30 < 180|

Above summary table is taken from the [National Building Code of Canada 2015](https://nrc.canada.ca/en/certifications-evaluations-standards/codes-canada/codes-canada-publications/national-building-code-canada-2015).

In [ ]:
VS30=450

--------------
### Ground Motion Prediction Equations

We combine the pertinent Ground Motion Prediction Equations (GMPE's) into Ground Shaking Intensity Models (GSIM's) using a logic tree structure. This is helpful when we have multiple GMPE's, possibly with different relative weighting. In this case, we will use the GSIM logic tree invoked by the 5th Generation National Seismic Hazard Map of Canada [see Atkinson and Adams, 2013, Halchuk et al., 2014, Adams et al., 2015, Allen et al., 2017, Adams and Halchuk, 2019]. It's supplied in the `what_you_need` folder.

In [ ]:
GSIM="what_you_need/gmmLT_nbcc2015_activefault.xml"

This GSIM has 3 GMPE's, representing the mean, upper, and lower limits on the shaking intensity. The GSIM's are developed for each tectonic environment, which we recognize with an acronym for naming:

|Region|Acronym|Tectonic Environment|
|:-----|:-----:|:-------------------|
|East|SC|Stable Crust|
|West|AC|Active Crust|
||AF|Active Fault|
||IS|Intraslab Shallow (30km)|
||ID|Intraslab Deep (55km)|
||SI|Subduction Interface|
|Either|SM|USGS Shakemap|

Because we're using a shallow event in the Georgia Strait near Vancouver, BC, we would use either Active Crust or Active Fault. The epicenter of our scenario event, however, is colocated on a fault that is known to have experienced seismicity in the last decades. Therefore, we've chosen Active Fault. 

------------
### Fragility Functions

For this exercise, we use a structural fragility file created by GEM, which relates the amount of shaking to the anticipated damage state of the building. These functions are based on HAZUS, and are undergoing updates to reflect Canadian building practices. The file is supplied for you in the `what_you_need` folder.

![alt text](http://www.st-risk.com/FragilityCurves.jpg "Fragility curves showing cumulative probability of damage versus spectral displacement")

In [ ]:
FRAG="what_you_need/structural_fragility.xml"

-----------------------------
-----------------------------
## Scenario Damage

### Build the Configuration/Initialization File

The snippet below will build your configuration file. You can see that it has a comment at the top that allows you to leave yourself a note about what was involved in this run. Below, you will specify the exposure model we just built. 

Next we'll specify the soil/site parameters, the reference to the earthquake scenario rupture we built, the ground shaking intensity model (GSIM), and the fragility function. 

Don't worry too much for now about the rest of the values. You're welcome to play around with these parameters, or just hit 'run' to build the default file.

In [ ]:
cat > configDamage.ini << EOF
[general]
description = Scenario damage calculation - generated by CheckMyHouseTutorial Jupyter Notebook - showing to Joost, Will, Murray, Drew.
calculation_mode = scenario_damage
random_seed = 113

[Exposure model]
exposure_file = $XML_OUTPUT

[site_params]
reference_vs30_type = inferred
reference_vs30_value = $VS30
reference_depth_to_2pt5km_per_sec = 5.0
reference_depth_to_1pt0km_per_sec = 100.0

[Rupture information]
rupture_model_file = $RUPTURE
rupture_mesh_spacing = 2

[Calculation parameters]
gsim_logic_tree_file = $GSIM 
truncation_level = 3.0
maximum_distance = 300.0
number_of_ground_motion_fields = 500

[fragility]
structural_fragility_file = $FRAG

[risk_calculation]
master_seed = 42 
EOF

----------
### Running Scenario Damage Calculator

Now you're ready to run it! The command below will ask OQ to process our configuration file, which calls to all the other files we've talked about. 

And how about a soundtrack for while it runs?

<a href="http://www.youtube.com/watch?feature=player_embedded&v=j6ZsqBqkQ-0&t=11" target="_blank"><img src="http://img.youtube.com/vi/j6ZsqBqkQ-0/0.jpg" 
alt="Sante Les Amis - Brazil" width="600" border="10" /></a>

In [ ]:
oq engine --run configDamage.ini

----------
### Exporting the Output
What damage state is your building in? Let's download the results in two ways. First, we will obtain the average results per asset (your house). This is the average number of buildings in each damage state, over all of the realizations we considered. In this case, it's like a probability of being in each damage state: none, slight, moderate, extensive, and complete. These damage states are described in detail in [this HAZUS Manual appendix](https://www.eeri.org/wp-content/uploads/Appendix-2-HAZUS-Damage-States.pdf). For reference, here it the description for a W1 building:    

   * __Slight Damage__: Small plaster or gypsum-board cracks at corners of door and window openings and wall-ceiling intersections; small cracks in masonry chimneys and masonry veneer.    
   * __Moderate Damage__: Large plaster or gypsum-board cracks at corners of door and window openings; small diagonal cracks across shear wall panels exhibited by small cracks in stucco and gypsum wall panels; large cracks in brick chimneys; toppling of tall masonry chimneys.     
   * __Extensive Damage__: Large diagonal cracks across shear wall panels or large cracks at plywood joints; permanent lateral movement of floors and roof; toppling of most brick chimneys; cracks in foundations; splitting of wood sill plates and/or slippage of structure over foundations; partial collapse of “room-over-garage” or other “soft-story” configurations; small foundations cracks.     
   * __Complete Damage__: Structure may have large permanent lateral displacement, may collapse, or be in imminent danger of collapse due to cripple wall failure or the failure of the lateral load resisting system; some structures may slip and fall off the foundations; large foundation cracks. Approximately 3% of the total area of W1 buildings with Complete damage is expected to be collapsed. 

In [ ]:
oq export dmg_by_asset -1

Now we'll download the results per event. An 'event' refers to a realization of the rupture, ground motion, and soil conditions within their uncertainty space. You may have noticed that we specified 500 realizations for this calculation, and a ground motion prediction equation logic tree with 3 entries. That's 1500 'events' to sample from. And in each event, OQ assigns your house to a damage state. That's why this output will have integer values instead of fractions. 

In [ ]:
oq export dmg_by_event -1

------------
------------

## Visualizing Outputs

You can look at your outputs in a Geopspatial Information System (GIS) tool like QGIS, in a simple plotting program like matplotlib, or with spreadsheet software like Microsoft Excel. QGIS is a particularly attractive option because it has an OQ plugin that's designed to allow for rapid assessment of outputs. You could also use a hybrid visualization tool like Tableau, although it is well beyond the scope today. 

For today, we'll just use a Python-based tool called `matplotib` to extract the information from our csv outputs. The snippet below creates a python script and runs it. It should spit out the probability of the most likely damage state for your house to be in, as well as a pie chart of the relative probability of all damage states. You can practice playing around with the plotting, using one of the two styles shown below. For more information about pie charts and donuts, check out [this helpful reference](https://matplotlib.org/3.1.1/gallery/pie_and_polar_charts/pie_and_donut_labels.html).

In [ ]:
cat > plotCheckMyHouse.py << EOF
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import glob

# Identify our selected output
fileName = "dmg_by_asset-rlz-001_???.csv"
file = glob.glob(fileName)

# Import it
data = pd.read_csv(str(file[0]))
dmg=data.iloc[0]['structural~no_damage_mean'],data.iloc[0]['structural~slight_mean'],data.iloc[0]['structural~moderate_mean'],data.iloc[0]['structural~extensive_mean'],data.iloc[0]['structural~complete_mean']
labels='None','Slight','Moderate','Extensive','Complete'

# Find most likely damage state and emphasize it
mostLikely=dmg.index(max(dmg))
explo = [0]*len(dmg); explo[mostLikely]=explo[mostLikely]+0.1; explo2 = tuple(explo)
print("You house has a "+str(100*dmg[mostLikely])+"% chance of being in Damage State "+str(labels[mostLikely]))

# Make a pie chart
#fig1, ax1 = plt.subplots()
#ax1.pie(dmg, labels=labels, explode=explo2, autopct='%1.1f%%', startangle=-70)
#ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
#plt.show()

# Make a donut chart
fig, ax = plt.subplots(figsize=(7, 3), subplot_kw=dict(aspect="equal"))
wedges, texts = ax.pie(dmg, wedgeprops=dict(width=0.5), startangle=-70)
bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")
for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                horizontalalignment=horizontalalignment, **kw)
ax.set_title("Probability of Different Damage States")
plt.show()

EOF

python3 plotCheckMyHouse.py

Be sure to close the figure window before trying to re-run this calculation. 

----------
## The End

Hopefully you've succesfully made it through, and have determined the probability of different damage states for your home in this scenario earthquake. You have also started thinking about the other datasets and models that are required for running OpenQuake, and experimented with plotting. Go celebrate your hard work!

![alt text](https://media.giphy.com/media/f9RzoxHizH72k15FKS/giphy.gif "Even Paul Rudd is proud of you!")